In [1]:
# pip install torch torchvision torchaudio --index-url https://download.pytorch.org/whl/cu117

Looking in indexes: https://download.pytorch.org/whl/cu117
Note: you may need to restart the kernel to use updated packages.


In [2]:
pip install matplotlib pillow opencv-python lxml

Note: you may need to restart the kernel to use updated packages.


In [3]:
pip install torch torchvision pillow numpy

Note: you may need to restart the kernel to use updated packages.


In [4]:
voc_root = r"D:\VOCdevkit\VOC2012"  # Change to your actual VOC2012 path

In [5]:
pip install --upgrade pillow

Note: you may need to restart the kernel to use updated packages.


In [6]:
import os
import glob

images_folder = r"D:\saliency\VOCdevkit\VOC2012\JPEGImages"  
print("Folder exists?", os.path.isdir(images_folder))

# Let's see what's inside the folder
print("Contents of JPEGImages folder:", os.listdir(images_folder))

# Now let's glob for .jpg
img_list = glob.glob(os.path.join(images_folder, '*.jpg'))
print("Found", len(img_list), "jpg files:")
for f in img_list[:5]:
    print("  ", f)


Folder exists? True
Contents of JPEGImages folder: ['2007_000027.jpg', '2007_000032.jpg', '2007_000033.jpg', '2007_000039.jpg', '2007_000042.jpg', '2007_000061.jpg', '2007_000063.jpg', '2007_000068.jpg', '2007_000121.jpg', '2007_000123.jpg', '2007_000129.jpg', '2007_000170.jpg', '2007_000175.jpg', '2007_000187.jpg', '2007_000241.jpg', '2007_000243.jpg', '2007_000250.jpg', '2007_000256.jpg', '2007_000272.jpg', '2007_000323.jpg', '2007_000332.jpg', '2007_000333.jpg', '2007_000346.jpg', '2007_000363.jpg', '2007_000364.jpg', '2007_000392.jpg', '2007_000423.jpg', '2007_000452.jpg', '2007_000464.jpg', '2007_000480.jpg', '2007_000491.jpg', '2007_000504.jpg', '2007_000515.jpg', '2007_000528.jpg', '2007_000529.jpg', '2007_000549.jpg', '2007_000559.jpg', '2007_000572.jpg', '2007_000584.jpg', '2007_000629.jpg', '2007_000636.jpg', '2007_000645.jpg', '2007_000648.jpg', '2007_000661.jpg', '2007_000663.jpg', '2007_000664.jpg', '2007_000676.jpg', '2007_000713.jpg', '2007_000720.jpg', '2007_000727.jpg'

In [7]:
import os
import glob

# CHANGE THIS to your actual VOC2012 path
voc_root = r"D:\saliency\VOCdevkit\VOC2012"

images_folder = os.path.join(voc_root, "JPEGImages")

print("Checking folder:", images_folder)
if not os.path.isdir(images_folder):
    print("[ERROR] JPEGImages folder not found. Check the path!")
    exit()

image_extensions = ["*.jpg", "*.jpeg", "*.png", "*.JPG", "*.JPEG", "*.PNG"]
image_files = []
for ext in image_extensions:
    image_files.extend(glob.glob(os.path.join(images_folder, ext)))

print("Found", len(image_files), "images in", images_folder)
if image_files:
    print("First 5 images:")
    for img in image_files[:5]:
        print("  ", img)
else:
    print("[WARNING] No images found. Check file extensions or subfolders.")


Checking folder: D:\saliency\VOCdevkit\VOC2012\JPEGImages
Found 34250 images in D:\saliency\VOCdevkit\VOC2012\JPEGImages
First 5 images:
   D:\saliency\VOCdevkit\VOC2012\JPEGImages\2007_000027.jpg
   D:\saliency\VOCdevkit\VOC2012\JPEGImages\2007_000032.jpg
   D:\saliency\VOCdevkit\VOC2012\JPEGImages\2007_000033.jpg
   D:\saliency\VOCdevkit\VOC2012\JPEGImages\2007_000039.jpg
   D:\saliency\VOCdevkit\VOC2012\JPEGImages\2007_000042.jpg


In [8]:
voc_root = r"D:\saliency\VOCdevkit\VOC2012"

In [10]:
import os
import glob
import xml.etree.ElementTree as ET

import torch
import numpy as np
from PIL import Image, ImageDraw, ImageFont
import matplotlib.pyplot as plt
from collections import defaultdict
from torchvision import models, transforms
from torchvision.ops import nms
from torchvision.models.detection import FasterRCNN_ResNet50_FPN_Weights

import random

# ------------------------------------------------------------------------------
# 1. Helper Functions
# ------------------------------------------------------------------------------
def parse_voc_annotation(xml_file):
    """
    Reads a VOC-style XML annotation file and returns a list of dicts:
      Each dict has:
        'name' -> class name (e.g., "person", "car", etc.)
        'bbox' -> [xmin, ymin, xmax, ymax]
    """
    tree = ET.parse(xml_file)
    root = tree.getroot()
    objects = []
    for obj in root.findall("object"):
        cls = obj.find("name").text.strip()
        bbox_xml = obj.find("bndbox")
        bbox = [
            float(bbox_xml.find("xmin").text),
            float(bbox_xml.find("ymin").text),
            float(bbox_xml.find("xmax").text),
            float(bbox_xml.find("ymax").text)
        ]
        objects.append({"name": cls, "bbox": bbox})
    return objects

def compute_iou(box1, box2):
    """
    Computes the Intersection over Union (IoU) of two bounding boxes.
    Box format: [xmin, ymin, xmax, ymax]
    """
    xA = max(box1[0], box2[0])
    yA = max(box1[1], box2[1])
    xB = min(box1[2], box2[2])
    yB = min(box1[3], box2[3])
    
    interW = max(0, xB - xA)
    interH = max(0, yB - yA)
    interArea = interW * interH

    box1Area = (box1[2] - box1[0]) * (box1[3] - box1[1])
    box2Area = (box2[2] - box2[0]) * (box2[3] - box2[1])

    if (box1Area + box2Area - interArea) == 0:
        return 0.0

    iou = interArea / float(box1Area + box2Area - interArea)
    return iou

def voc_ap(rec, prec):
    """
    Computes the Average Precision (AP) in a VOC-style manner:
    integrates over the precision-recall curve.
    """
    mrec = np.concatenate(([0.], rec, [1.]))
    mpre = np.concatenate(([0.], prec, [0.]))

    # Make precision monotonically decreasing
    for i in range(mpre.size - 1, 0, -1):
        mpre[i - 1] = max(mpre[i - 1], mpre[i])

    # Identify points where recall changes
    i_list = np.where(mrec[1:] != mrec[:-1])[0]
    ap = 0.0
    for i in i_list:
        ap += (mrec[i + 1] - mrec[i]) * mpre[i + 1]
    return ap

# ------------------------------------------------------------------------------
# 2. Paths and Model Setup
# ------------------------------------------------------------------------------
# Use your actual VOC2012 location (keep the original path)
voc_root = r"D:\saliency\VOCdevkit\VOC2012"

annotations_folder = os.path.join(voc_root, "Annotations")
images_folder = os.path.join(voc_root, "JPEGImages")

output_folder = os.path.join(voc_root, "output_detections")
os.makedirs(output_folder, exist_ok=True)

# Force CPU if needed
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

# Load pre-trained Faster R-CNN (only FasterRCNN is used here)
weights = FasterRCNN_ResNet50_FPN_Weights.COCO_V1
model = models.detection.fasterrcnn_resnet50_fpn(weights=weights)
model.eval()
model.to(device)  # Move model to GPU if available

# Transformation for input images
transform = transforms.Compose([
    transforms.ToTensor(),
])

# ------------------------------------------------------------------------------
# 3. Gather Image Files
# ------------------------------------------------------------------------------
# We'll read all JPG/JPEG/PNG images in the JPEGImages folder
image_extensions = ["*.jpg", "*.jpeg", "*.png", "*.JPG", "*.JPEG", "*.PNG"]
image_files = []
for ext in image_extensions:
    image_files.extend(glob.glob(os.path.join(images_folder, ext)))

if not image_files:
    print("[ERROR] No image files found in the JPEGImages folder.")
    print("Check your path or file extensions.")
    exit()

# ------------------------------------------------------------------------------
# 3.1 Take a subset of the dataset (adjust the ratio as needed)
# ------------------------------------------------------------------------------
random.shuffle(image_files)
num_images = int(0.3 * len(image_files))
image_files = image_files[:num_images]

print(f"Found {len(image_files)} images (subset of total).")
print("Starting inference...\n")

# ------------------------------------------------------------------------------
# 4. Detection & Evaluation Settings
# ------------------------------------------------------------------------------
CONFIDENCE_THRESHOLD = 0.2   # Minimum score to keep a detection
NMS_THRESHOLD = 0.7          # Non-Max Suppression threshold
MIN_OBJECT_SIZE = 20         # Ignore boxes smaller than this
IOU_THRESHOLD = 0.5          # IoU >= 0.5 => True Positive

# ------------------------------------------------------------------------------
# 5. Data Structures for Storing Results
# ------------------------------------------------------------------------------
# all_annotations: { class_name: { image_id: [ {bbox, detected}, ... ] } }
all_annotations = defaultdict(lambda: defaultdict(list))

# all_detections: { class_name: [ {image_id, confidence, bbox}, ... ] }
all_detections = defaultdict(list)

# Global metrics (Precision, Recall, F1, etc.)
global_TP = 0
global_FP = 0
global_FN = 0

# ------------------------------------------------------------------------------
# 6. COCO to VOC Class Mapping
# ------------------------------------------------------------------------------
COCO_INSTANCE_CATEGORY_NAMES = [
    "__background__", "person", "bicycle", "car", "motorcycle", "airplane", "bus",
    "train", "truck", "boat", "traffic light", "fire hydrant", "N/A", "stop sign",
    "parking meter", "bench", "bird", "cat", "dog", "horse", "sheep", "cow",
    "elephant", "bear", "zebra", "giraffe", "N/A", "backpack", "umbrella",
    "handbag", "tie", "suitcase", "frisbee", "skis", "snowboard", "sports ball",
    "kite", "baseball bat", "baseball glove", "skateboard", "surfboard",
    "tennis racket", "bottle", "wine glass", "cup", "fork", "knife", "spoon",
    "bowl", "banana", "apple", "sandwich", "orange", "broccoli", "carrot",
    "hot dog", "pizza", "donut", "cake", "chair", "couch", "potted plant",
    "bed", "dining table", "toilet", "tv", "laptop", "mouse", "remote",
    "keyboard", "cell phone", "microwave", "oven", "toaster", "sink",
    "refrigerator", "book", "clock", "vase", "scissors", "teddy bear",
    "hair drier", "toothbrush"
]

COCO_TO_VOC = {
    "person": "person",
    "bicycle": "bicycle",
    "car": "car",
    "motorcycle": "motorbike",      # "motorcycle" -> "motorbike"
    "airplane": "aeroplane",         # "airplane" -> "aeroplane"
    "bus": "bus",
    "train": "train",
    "boat": "boat",
    "bird": "bird",
    "cat": "cat",
    "dog": "dog",
    "horse": "horse",
    "sheep": "sheep",
    "cow": "cow",
    "bottle": "bottle",
    "chair": "chair",
    "couch": "sofa",               # "couch" in COCO => "sofa" in VOC
    "potted plant": "pottedplant", # "potted plant" => "pottedplant"
    "dining table": "diningtable", # "dining table" => "diningtable"
    "tv": "tvmonitor"              # "tv" => "tvmonitor"
}

# ------------------------------------------------------------------------------
# 7. Preload Annotations
# ------------------------------------------------------------------------------
for img_path in image_files:
    image_id = os.path.splitext(os.path.basename(img_path))[0]
    xml_path = os.path.join(annotations_folder, f"{image_id}.xml")
    if os.path.exists(xml_path):
        objs = parse_voc_annotation(xml_path)
        for obj in objs:
            cls_name = obj["name"]
            all_annotations[cls_name][image_id].append({"bbox": obj["bbox"], "detected": False})

# ------------------------------------------------------------------------------
# 8. Run Inference & Collect Predictions
# ------------------------------------------------------------------------------
for i, img_file in enumerate(image_files):
    image_id = os.path.splitext(os.path.basename(img_file))[0]
    image = Image.open(img_file).convert("RGB")

    # Convert PIL to tensor
    img_tensor = transform(image).unsqueeze(0).to(device)

    # Model Inference
    with torch.no_grad():
        predictions = model(img_tensor)[0]

    scores = predictions["scores"].cpu().numpy()
    boxes = predictions["boxes"].cpu().numpy()
    labels = predictions["labels"].cpu().numpy()  # COCO label IDs

    # Filter by confidence and box size
    filtered_boxes = []
    filtered_scores = []
    filtered_labels = []
    for j in range(len(scores)):
        score = scores[j]
        box = boxes[j]
        width = box[2] - box[0]
        height = box[3] - box[1]
        if score >= CONFIDENCE_THRESHOLD and width > MIN_OBJECT_SIZE and height > MIN_OBJECT_SIZE:
            filtered_boxes.append(box)
            filtered_scores.append(score)
            filtered_labels.append(labels[j])

    # Apply Non-Max Suppression (NMS)
    if filtered_boxes:
        keep_indices = nms(
            torch.tensor(filtered_boxes),
            torch.tensor(filtered_scores),
            NMS_THRESHOLD
        )
        filtered_boxes = [filtered_boxes[k] for k in keep_indices]
        filtered_scores = [filtered_scores[k] for k in keep_indices]
        filtered_labels = [filtered_labels[k] for k in keep_indices]

    print(f"[{i+1}/{len(image_files)}] {image_id}: {len(filtered_boxes)} final detections")

    # Map each detection's COCO label to its VOC counterpart (if it exists)
    for box, score, label_id in zip(filtered_boxes, filtered_scores, filtered_labels):
        if label_id < len(COCO_INSTANCE_CATEGORY_NAMES):
            coco_class_name = COCO_INSTANCE_CATEGORY_NAMES[label_id]
        else:
            continue

        if coco_class_name in COCO_TO_VOC:
            class_name = COCO_TO_VOC[coco_class_name]
        else:
            continue

        all_detections[class_name].append({
            "image_id": image_id,
            "confidence": score,
            "bbox": box
        })

    # Visualization for the first 5 images
    if i < 5:
        draw_image = image.copy()
        draw = ImageDraw.Draw(draw_image)
        try:
            font = ImageFont.truetype("arial.ttf", 15)
        except IOError:
            font = ImageFont.load_default()

        for box, score, label_id in zip(filtered_boxes, filtered_scores, filtered_labels):
            if label_id < len(COCO_INSTANCE_CATEGORY_NAMES):
                coco_class_name = COCO_INSTANCE_CATEGORY_NAMES[label_id]
                if coco_class_name in COCO_TO_VOC:
                    voc_class_name = COCO_TO_VOC[coco_class_name]
                else:
                    continue
            else:
                continue

            draw.rectangle(box, outline="red", width=2)
            text = f"{voc_class_name}: {score:.2f}"
            text_bbox = draw.textbbox((0, 0), text, font=font)
            text_w = text_bbox[2] - text_bbox[0]
            text_h = text_bbox[3] - text_bbox[1]
            text_loc = (box[0], box[1] - text_h)
            draw.rectangle([text_loc, (box[0] + text_w, box[1])], fill="red")
            draw.text(text_loc, text, fill="white", font=font)

        out_path = os.path.join(output_folder, f"{image_id}_det.jpg")
        draw_image.save(out_path)
        print(f"Saved visualization: {out_path}")

# ------------------------------------------------------------------------------
# 9. Compute Average Precision (AP) and Global Metrics
# ------------------------------------------------------------------------------
aps = {}

for cls_name, detections in all_detections.items():
    detections = sorted(detections, key=lambda x: x["confidence"], reverse=True)

    TP_array = np.zeros(len(detections))
    FP_array = np.zeros(len(detections))

    total_gt = 0
    if cls_name in all_annotations:
        for img_id in all_annotations[cls_name]:
            total_gt += len(all_annotations[cls_name][img_id])

    for idx, det in enumerate(detections):
        image_id = det["image_id"]
        box_pred = det["bbox"]

        if cls_name in all_annotations and image_id in all_annotations[cls_name]:
            max_iou = 0
            assigned_gt = None
            for gt in all_annotations[cls_name][image_id]:
                iou = compute_iou(box_pred, gt["bbox"])
                if iou > max_iou:
                    max_iou = iou
                    assigned_gt = gt
            
            if max_iou >= IOU_THRESHOLD and assigned_gt is not None:
                if not assigned_gt["detected"]:
                    TP_array[idx] = 1
                    assigned_gt["detected"] = True
                else:
                    FP_array[idx] = 1
            else:
                FP_array[idx] = 1
        else:
            FP_array[idx] = 1

    cum_TP = np.cumsum(TP_array)
    cum_FP = np.cumsum(FP_array)
    rec = cum_TP / (total_gt + 1e-6)
    prec = cum_TP / (cum_TP + cum_FP + 1e-6)

    ap = voc_ap(rec, prec)
    aps[cls_name] = ap
    print(f"AP for class '{cls_name}': {ap:.4f}")

    TP_class = int(cum_TP[-1])
    FP_class = int(cum_FP[-1])
    FN_class = total_gt - TP_class

    global_TP += TP_class
    global_FP += FP_class
    global_FN += FN_class

mAP = np.mean(list(aps.values())) if aps else 0.0
print(f"\nMean Average Precision (mAP): {mAP:.4f}")

# ------------------------------------------------------------------------------
# 10. Compute Global Precision, Recall, F1, and Accuracy
# ------------------------------------------------------------------------------
precision_global = global_TP / float(global_TP + global_FP + 1e-6)
recall_global = global_TP / float(global_TP + global_FN + 1e-6)
f1_global = 2.0 * (precision_global * recall_global) / (precision_global + recall_global + 1e-6)
accuracy_global = global_TP / float(global_TP + global_FP + global_FN + 1e-6)

print("\n===== Global Metrics (across all classes & images) =====")
print(f"True Positives (TP): {global_TP}")
print(f"False Positives (FP): {global_FP}")
print(f"False Negatives (FN): {global_FN}")
print(f"Precision: {precision_global:.4f}")
print(f"Recall:    {recall_global:.4f}")
print(f"F1-score:  {f1_global:.4f}")
print(f"Accuracy:  {accuracy_global:.4f}")


Using device: cuda
Found 10275 images (subset of total).
Starting inference...

[1/10275] 2008_002419: 17 final detections
Saved visualization: D:\saliency\VOCdevkit\VOC2012\output_detections\2008_002419_det.jpg
[2/10275] 2012_002611: 1 final detections
Saved visualization: D:\saliency\VOCdevkit\VOC2012\output_detections\2012_002611_det.jpg
[3/10275] 2011_000102: 6 final detections
Saved visualization: D:\saliency\VOCdevkit\VOC2012\output_detections\2011_000102_det.jpg
[4/10275] 2012_004158: 21 final detections
Saved visualization: D:\saliency\VOCdevkit\VOC2012\output_detections\2012_004158_det.jpg
[5/10275] 2008_007319: 4 final detections
Saved visualization: D:\saliency\VOCdevkit\VOC2012\output_detections\2008_007319_det.jpg
[6/10275] 2009_000218: 2 final detections
[7/10275] 2009_004112: 6 final detections
[8/10275] 2008_000558: 1 final detections
[9/10275] 2010_001061: 1 final detections
[10/10275] 2012_003068: 5 final detections
[11/10275] 2008_005838: 6 final detections
[12/10275

In [1]:
import torch
import torchvision
import torchvision.transforms.functional as F
from torchvision.models.detection.faster_rcnn import FastRCNNPredictor
from torch.utils.data import DataLoader, Subset
import random
import math
from PIL import Image, ImageOps
from torchvision.datasets import VOCDetection
from tqdm import tqdm

class Compose(object):
    def __init__(self, transforms):
        self.transforms = transforms

    def __call__(self, image, target):
        for t in self.transforms:
            image, target = t(image, target)
        return image, target

class ToTensor(object):
    def __call__(self, image, target):
        image = F.to_tensor(image)
        return image, target

class RandomHorizontalFlip(object):
    def __init__(self, p=0.5):
        self.p = p

    def __call__(self, image, target):
        if random.random() < self.p:
            image = ImageOps.mirror(image)
            ann = target['annotation']
            width = image.size[0]
            def flip_box(bbox):
                xmin = float(bbox['xmin'])
                xmax = float(bbox['xmax'])
                bbox['xmin'] = str(width - xmax)
                bbox['xmax'] = str(width - xmin)
                return bbox
            
            if 'object' in ann:
                objects = ann['object']
                if isinstance(objects, list):
                    for obj in objects:
                        obj['bndbox'] = flip_box(obj['bndbox'])
                else:
                    ann['object']['bndbox'] = flip_box(ann['object']['bndbox'])
            target['annotation'] = ann
        return image, target

def get_transform(train):
    transforms = []
    if train:
        transforms.append(RandomHorizontalFlip(0.5))
    transforms.append(ToTensor())
    return Compose(transforms)

VOC_CLASSES = [
    '__background__', 'aeroplane', 'bicycle', 'bird', 'boat', 'bottle',
    'bus', 'car', 'cat', 'chair', 'cow', 'diningtable', 'dog', 'horse',
    'motorbike', 'person', 'pottedplant', 'sheep', 'sofa', 'train', 'tvmonitor'
]

def voc_class_to_int(class_name):
    return VOC_CLASSES.index(class_name)

def parse_voc_annotation(target):
    ann = target['annotation']
    objects = ann.get('object', [])
    if not isinstance(objects, list):
        objects = [objects]
    
    boxes = []
    labels = []
    for obj in objects:
        bbox = obj['bndbox']
        box = [
            float(bbox['xmin']),
            float(bbox['ymin']),
            float(bbox['xmax']),
            float(bbox['ymax'])
        ]
        boxes.append(box)
        labels.append(voc_class_to_int(obj['name']))
    
    return {
        'boxes': torch.as_tensor(boxes, dtype=torch.float32),
        'labels': torch.as_tensor(labels, dtype=torch.int64)
    }

def get_model(num_classes):
    model = torchvision.models.detection.fasterrcnn_resnet50_fpn(weights=torchvision.models.detection.FasterRCNN_ResNet50_FPN_Weights.COCO_V1)
    in_features = model.roi_heads.box_predictor.cls_score.in_features
    model.roi_heads.box_predictor = FastRCNNPredictor(in_features, num_classes)
    return model

def collate_fn(batch):
    return tuple(zip(*batch))

# Full datasets
full_train_dataset = VOCDetection(
    root='VOCdevkit', year='2007', image_set='train',
    download=True, transforms=get_transform(train=True)
)
full_val_dataset = VOCDetection(
    root='VOCdevkit', year='2007', image_set='val',
    download=False, transforms=get_transform(train=False)
)

# Take 15% of datasets
train_size = math.ceil(0.15 * len(full_train_dataset))
val_size = math.ceil(0.15 * len(full_val_dataset))

train_indices = random.sample(range(len(full_train_dataset)), train_size)
val_indices = random.sample(range(len(full_val_dataset)), val_size)

train_dataset = Subset(full_train_dataset, train_indices)
val_dataset = Subset(full_val_dataset, val_indices)

# Dataloaders
train_loader = DataLoader(train_dataset, batch_size=4, shuffle=True, collate_fn=collate_fn)
val_loader = DataLoader(val_dataset, batch_size=1, shuffle=False, collate_fn=collate_fn)

device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')
num_classes = len(VOC_CLASSES)
model = get_model(num_classes)
model.to(device)

params = [p for p in model.parameters() if p.requires_grad]
optimizer = torch.optim.SGD(params, lr=0.001, momentum=0.9, weight_decay=0.0001)
lr_scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size=3, gamma=0.1)

num_epochs = 5
model.train()

for epoch in range(num_epochs):
    epoch_loss = 0.0
    progress_bar = tqdm(train_loader, desc=f"Epoch {epoch+1}/{num_epochs}", leave=True)
    
    for images, targets in progress_bar:
        images = [img.to(device) for img in images]
        targets_converted = [parse_voc_annotation(t) for t in targets]
        targets_converted = [{k: v.to(device) for k, v in target.items()} for target in targets_converted]
        
        optimizer.zero_grad()
        loss_dict = model(images, targets_converted)
        losses = sum(loss for loss in loss_dict.values())
        loss_value = losses.item()
        
        losses.backward()
        optimizer.step()
        
        epoch_loss += loss_value
        progress_bar.set_postfix(loss=loss_value)
    
    lr_scheduler.step()
    print(f"Epoch {epoch+1} completed - Avg Loss: {epoch_loss / len(train_loader):.4f}")

print("Training complete!")


Epoch 1/5: 100%|███████████████████████████████████████████████████████████| 94/94 [24:57<00:00, 15.93s/it, loss=0.387]


Epoch 1 completed - Avg Loss: 0.8758


Epoch 2/5: 100%|███████████████████████████████████████████████████████████| 94/94 [24:42<00:00, 15.77s/it, loss=0.569]


Epoch 2 completed - Avg Loss: 0.6188


Epoch 3/5: 100%|███████████████████████████████████████████████████████████| 94/94 [24:37<00:00, 15.72s/it, loss=0.356]


Epoch 3 completed - Avg Loss: 0.5155


Epoch 4/5: 100%|███████████████████████████████████████████████████████████| 94/94 [24:24<00:00, 15.58s/it, loss=0.359]


Epoch 4 completed - Avg Loss: 0.4489


Epoch 5/5: 100%|███████████████████████████████████████████████████████████| 94/94 [24:05<00:00, 15.38s/it, loss=0.676]

Epoch 5 completed - Avg Loss: 0.4366
Training complete!


In [2]:
import torchvision.ops as ops

def calculate_iou(box1, box2):
    """Calculate IoU of two boxes."""
    box1 = torch.tensor(box1).unsqueeze(0)
    box2 = torch.tensor(box2).unsqueeze(0)
    return ops.box_iou(box1, box2).item()

def evaluate(model, data_loader, device, iou_threshold=0.5):
    model.eval()
    correct = 0
    total = 0

    with torch.no_grad():
        for images, targets in tqdm(data_loader, desc="Evaluating"):
            images = [img.to(device) for img in images]
            outputs = model(images)

            for output, target in zip(outputs, targets):
                gt = parse_voc_annotation(target)
                gt_boxes = gt['boxes'].cpu()
                gt_labels = gt['labels'].cpu()

                pred_boxes = output['boxes'].cpu()
                pred_labels = output['labels'].cpu()

                for gt_box, gt_label in zip(gt_boxes, gt_labels):
                    total += 1
                    match_found = False
                    for pred_box, pred_label in zip(pred_boxes, pred_labels):
                        iou = calculate_iou(gt_box, pred_box)
                        if iou >= iou_threshold and pred_label == gt_label:
                            match_found = True
                            break
                    if match_found:
                        correct += 1

    accuracy = 100 * correct / total if total > 0 else 0
    print(f"Validation Accuracy: {accuracy:.2f}% ({correct}/{total} correct matches)")

# 🔹 Run evaluation after training
evaluate(model, val_loader, device)


C:\Users\FireFly\AppData\Local\Temp\ipykernel_16060\368014669.py:5: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  box1 = torch.tensor(box1).unsqueeze(0)
C:\Users\FireFly\AppData\Local\Temp\ipykernel_16060\368014669.py:6: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  box2 = torch.tensor(box2).unsqueeze(0)
Evaluating: 100%|████████████████████████████████████████████████████████████████████| 377/377 [03:47<00:00,  1.66it/s]

Validation Accuracy: 88.05% (877/996 correct matches)
